In [3]:
from pyannote.audio import Pipeline
import os

api_key = os.getenv("HUGGING_TOKEN")

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=api_key
)

if torch.cuda.is_available():
    pipeline.to(torch.device("cuda"))
    print("cuda is available")
else:
    print('cuda is not available')


Could not download 'pyannote/segmentation-3.0' model.
It might be because the model is private or gated so make
sure to authenticate. Visit https://hf.co/settings/tokens to
create your access token and retry with:

   >>> Model.from_pretrained('pyannote/segmentation-3.0',
   ...                       use_auth_token=YOUR_AUTH_TOKEN)

If this still does not work, it might be because the model is gated:
visit https://hf.co/pyannote/segmentation-3.0 to accept the user conditions.


AttributeError: 'NoneType' object has no attribute 'eval'